In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_by_zone_sql").getOrCreate()

In [3]:
trip_data = spark.read.csv(r"C:\Users\woals\data-engineering\01-spark\data\tripdata_2020_03.csv", inferSchema = True, header = True)
zone_data = spark.read.csv(r"C:\Users\woals\data-engineering\01-spark\data\taxi+_zone_lookup.csv", inferSchema = True, header = True)

In [4]:
trip_data.show(5)

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [5]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [6]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [7]:
spark.sql("select * from trip_data limit 5").show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [8]:
spark.sql("select * from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [9]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID) \
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2437394|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735765|
|Staten Island| 178818|
|    Manhattan|4953147|
|        Bronx|2086597|
+-------------+-------+



In [10]:
spark.sql("select borough, count(*) as trips from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID) \
group by borough").show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2468416|
|          EWR|  65066|
|      Unknown| 387760|
|     Brooklyn|3696684|
|Staten Island| 177727|
|    Manhattan|4553783|
|        Bronx|2043492|
+-------------+-------+



In [11]:
spark.sql("SELECT zone_data.Zone, count(*) AS trips\
 FROM trip_data JOIN zone_data ON trip_data.PULocationID = zone_data.LocationID \
 WHERE trip_data.hvfhs_license_num = 'HV0003' \
 GROUP BY zone_data.Zone order by trips desc").show()

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|163091|
|       East New York|134198|
|         JFK Airport|114182|
|        East Village|112017|
|      Bushwick South|110150|
|Central Harlem North|108070|
|   LaGuardia Airport|104119|
|Washington Height...| 97324|
|Flatbush/Ditmas Park| 95724|
|            Canarsie| 94484|
|TriBeCa/Civic Center| 94155|
|             Astoria| 92676|
|             Bedford| 90352|
|      Midtown Center| 90262|
|  Stuyvesant Heights| 88749|
|            Union Sq| 88372|
|Times Sq/Theatre ...| 86871|
|Prospect-Lefferts...| 84347|
|         Brownsville| 82764|
|Mott Haven/Port M...| 82396|
+--------------------+------+
only showing top 20 rows



In [12]:
spark.sql("SELECT zone_data.Zone, count(*) AS trips\
 FROM trip_data JOIN zone_data ON trip_data.PULocationID = zone_data.LocationID \
 WHERE trip_data.hvfhs_license_num = 'HV0003' \
 GROUP BY zone_data.Zone order by trips desc").explain(True)

== Parsed Logical Plan ==
'Sort ['trips DESC NULLS LAST], true
+- 'Aggregate ['zone_data.Zone], ['zone_data.Zone, 'count(1) AS trips#474]
   +- 'Filter ('trip_data.hvfhs_license_num = HV0003)
      +- 'Join Inner, ('trip_data.PULocationID = 'zone_data.LocationID)
         :- 'UnresolvedRelation [trip_data], [], false
         +- 'UnresolvedRelation [zone_data], [], false

== Analyzed Logical Plan ==
Zone: string, trips: bigint
Sort [trips#474L DESC NULLS LAST], true
+- Aggregate [Zone#86], [Zone#86, count(1) AS trips#474L]
   +- Filter (hvfhs_license_num#18 = HV0003)
      +- Join Inner, (PULocationID#25 = LocationID#84)
         :- SubqueryAlias trip_data
         :  +- View (`trip_data`, [_c0#17,hvfhs_license_num#18,dispatching_base_num#19,originating_base_num#20,request_datetime#21,on_scene_datetime#22,pickup_datetime#23,dropoff_datetime#24,PULocationID#25,DOLocationID#26,trip_miles#27,trip_time#28,base_passenger_fare#29,tolls#30,bcf#31,sales_tax#32,congestion_surcharge#33,airport_f